In [25]:
import cv2

cap = cv2.VideoCapture('Plaid Apple.webm')

# Read the first frame
ret, prev_2frame = cap.read()
if not ret:
    exit()
prev_2gray = cv2.cvtColor(prev_2frame, cv2.COLOR_BGR2GRAY)

ret, prev_1frame = cap.read()
if not ret:
    exit()
prev_1gray = cv2.cvtColor(prev_1frame, cv2.COLOR_BGR2GRAY)
#prev_gray = cv2.GaussianBlur(prev_gray, (21, 21), 0)  # Reduce noise

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale + blur
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Compute absolute difference between frames
    #frame_diff = np.add(cv2.absdiff(prev_2gray, gray), cv2.absdiff(prev_1gray, gray))
    frame_diff=(cv2.absdiff(prev_1gray, gray))
    _, motion_mask = cv2.threshold(frame_diff, 100, 255, cv2.THRESH_TOZERO)  # Adjust threshold

    # Save or display the motion mask
    cv2.imshow('Motion Mask', motion_mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    prev_2gray = prev_1gray
    prev_1gray=gray

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from qreader import QReader
from tqdm.auto import tqdm
from itertools import combinations
import numpy as np


cap = cv2.VideoCapture("Plaid Apple.webm")
#video is trimmed from 3:30 to 4:10
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )



def stackframes(start, length=13):
    frames=[]
    cap.set(cv2.CAP_PROP_POS_FRAMES, start)
    for i in range(length):
        ret, f=cap.read()
        #f=cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        if f is None:
            
            frames.append( np.zeros_like(frames[0]))
        else:
            frames.append(f)
    return frames

def diff(stack):
    diff=[]
    
    for i,j in combinations(stack, 2):
       
        diff.append(cv2.absdiff(i, j))

    return np.mean(diff, axis=0)


qreader=QReader(model_size="l", min_confidence=0.1)
prev=None
with open("ans.txt", "w") as f:
    for i in tqdm(range(175, length, 3)):
        
        stack=stackframes(i)
        if not stack:
            break
        delta=diff(stack)
        #im=delta.astype(np.uint8)
        #print(np.shape(delta), np.shape(stack))
        im=cv2.cvtColor(delta.astype(np.uint8), cv2.COLOR_BGR2GRAY)
        
        im1=nlm_denoised = cv2.fastNlMeansDenoising(im, None, h=10, templateWindowSize=7, searchWindowSize=21)
        im2 = cv2.medianBlur(im, 5)
        im3 = cv2.medianBlur(im, 7)

        
        
        q1=set(qreader.detect_and_decode(im1))
        q2=set(qreader.detect_and_decode(im2))
        q3=set(qreader.detect_and_decode(im3))
        
        
        s=q1.union(q2).union(q3)
     
        s=s - set([None])
        #print(s)
     
        if not s:
            continue
            #cv2.imwrite("qrcodes/idk"+str(i)+".png",im)
            
       
        elif s !=prev:
            prev=s
            print(i, s)
            print(i, " ".join(s), file=f, flush=True)

f.close()
#PCTF{i_spy_with_my_little_eye_persistence_of_vision_c32392f3e83c2741a5a3}